In [18]:
# Initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
from Constants import Constants
from PreprocessingTools import CustomerNameCleaningFunctions
from PreprocessingTools import DateCleaningFunctions
from PreprocessingTools import SpecializedDateCleaningFunctions

pn.extension()

%matplotlib inline

In [19]:
# Build tools
debug_level = 0
constants = Constants()
tool_customer = CustomerNameCleaningFunctions(debug_level)
tool_date = DateCleaningFunctions(debug_level)
tool_special_date = SpecializedDateCleaningFunctions(debug_level)


In [20]:
# Read anonymized data
atlas           = pd.read_csv(constants.ANON_ATLAS_FILE_PATH, index_col="Customers")
forecast        = pd.read_csv(constants.ANON_FORECAST_DATA_FILE_PATH, index_col="Organization Name")
revenue2020     = pd.read_csv(constants.ANON_REVENUE2020_FILE_PATH, index_col="Name")
revenue2020A    = pd.read_csv(constants.ANON_REVENUE2020A_FILE_PATH, index_col="Payee Name")

In [21]:
# Clean up ATLAS data

## Invoice Date
atlas["Invoice Date"]       = tool_date.cleanup_date_string_list(atlas["Invoice Date"])

## Invoice Amount
atlas["Invoice Amount"]      = tool_date.cleanup_dollar_string_list(atlas["Invoice Amount"])

## Dates of service
dates_of_service            = tool_special_date.extract_subscription_dates_list(atlas["Dates of service "])
atlas["Service Start"]      = dates_of_service[0]
atlas["Service End"]        = dates_of_service[1]
atlas.drop(columns=["Dates of service "], inplace=True)

atlas.head()

,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Service Start,Service End
Customers,,,,,,,
University 1,2015-03-20 00:00:00-04:00,ATLAS 315,72000.0,1 Year,4700-0-00-00000-18-0000,2015-03-18 00:00:00-04:00,2016-06-30 00:00:00-04:00
University 102,2015-05-28 00:00:00-04:00,AJ501,3500.0,1 Year,4700-0-00-00000-16-0000,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00
University 3,2015-06-23 00:00:00-04:00,AJ502,3500.0,1 Year,4700-0-00-00000-17-0000,2015-06-10 00:00:00-04:00,2016-06-30 00:00:00-04:00
University 4,2015-06-26 00:00:00-04:00,AJ503,6500.0,1 Year,4700-0-00-00000-32-0000,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00
University 5,2015-10-07 00:00:00-04:00,AJ504,750.0,1 Year,4700-0-00-00000-20-0000,2015-10-05 00:00:00-04:00,2016-09-30 00:00:00-04:00
